In [1]:
import os
import sys
sys.path.append('../')
from pulpo import pulpo

import pandas as pd
pd.set_option('display.max_colwidth', None)

### Setup the Worker

In [2]:
project = "pulpo-ammonia"
databases = ["nc-inventories-ei310-all", "ecoinvent-3.10-cutoff"]
methods = "('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')"

In [3]:
# Substitute with your working directory of choice
notebook_dir = os.path.dirname(os.getcwd())
directory = os.path.join(notebook_dir, 'data')

# Substitute with your GAMS path
GAMS_PATH = r"C:\APPS\GAMS\win64\40.1\gams.exe"

In [4]:
pulpo_worker = pulpo.PulpoOptimizer(project, databases, methods, directory)
pulpo_worker.intervention_matrix="ecoinvent-3.10-biosphere"

In [5]:
pulpo_worker.get_lci_data()

### Get the Choices

In [6]:
choices_biogas = [
    "anaerobic digestion of animal manure, with biogenic carbon uptake",
    "anaerobic digestion of agricultural residues, with biogenic carbon uptake",
    "treatment of sewage sludge by anaerobic digestion, cut-off with biogenic carbon uptake",
    "treatment of industrial wastewater by anaerobic digestion, cut-off with biogenic carbon uptake",
    "treatment of biowaste by anaerobic digestion, cut-off with biogenic carbon uptake",
    "anaerobic digestion of sequential crop, with biogenic carbon uptake"
]

choices_hydrogen = [
    "hydrogen production, biomass gasification",
    "hydrogen production, biomass gasification, with CCS",
    "hydrogen production, steam methane reforming of biomethane",
    "hydrogen production, steam methane reforming of biomethane, with CCS",
    "hydrogen production, steam methane reforming of natural gas, with CCS",
    "hydrogen production, PEM electrolysis, green",
    "green hydrogen",
    "hydrogen production, plastics gasification",
    "hydrogen production, plastics gasification, with CCS"
]

choices_heat = [
    "heat from biomethane",
    "heat from biomethane, with CCS",
    "heat from hydrogen",
    "heat from natural gas, with CCS"
]

choices_ammonia = [
    "ammonia production, steam methane reforming of biomethane",
    "ammonia production, steam methane reforming of biomethane, with CCS",
    "ammonia production, steam methane reforming of natural gas, with CCS",
    "ammonia production, from nitrogen and hydrogen"
]

choices_biomethane = [
    "biogas upgrading to biomethane, chemical scrubbing",
    "biogas upgrading to biomethane, chemical scrubbing w/ CCS",
    "biogas upgrading to biomethane, membrane",
    "biogas upgrading to biomethane, membrane w/ CCS",
    "biogas upgrading to biomethane, pressure swing adsorption",
    "biogas upgrading to biomethane, pressure swing adsorption w/ CCS",
    "biogas upgrading to biomethane, water scrubbing",
    "biogas upgrading to biomethane, water scrubbing w/ CCS"
]

In [7]:
# Retrieve activities for each category
biogas_activities = pulpo_worker.retrieve_activities(activities=choices_biogas)
hydrogen_activities = pulpo_worker.retrieve_activities(activities=choices_hydrogen)
heat_activities = pulpo_worker.retrieve_activities(activities=choices_heat)
biomethane_activities = pulpo_worker.retrieve_activities(activities=choices_biomethane)

ammonia_activities = pulpo_worker.retrieve_activities(activities=choices_ammonia)
# Add BAU Ammonia from ecoinvent as choice
ammonia_activities.append(pulpo_worker.retrieve_activities(reference_products="ammonia, anhydrous, liquid", activities="ammonia production, steam reforming, liquid", locations="RER w/o RU")[0])

In [8]:
choices = {
    "biogas": {x: 1e10 for x in biogas_activities},
    "hydrogen": {x: 1e10 for x in hydrogen_activities},
    "heat": {x: 1e10 for x in heat_activities},
    "biomethane": {x: 1e10 for x in biomethane_activities},
    "ammonia": {x: 1e10 for x in ammonia_activities},
}

### Demand

In [9]:
ammonia_market = pulpo_worker.retrieve_activities(activities="new market for ammonia")

In [10]:
demand = {ammonia_market[0]: 1}

### Instantiate and Solve

In [11]:
instance = pulpo_worker.instantiate(choices=choices, demand=demand)

Creating Instance
Instance created


In [12]:
results = pulpo_worker.solve(GAMS_PATH=GAMS_PATH)

GAMS solvers library availability: True
Solver path: C:\APPS\GAMS\win64\40.1\gams.exe

GAMS WORKING DIRECTORY: C:\Users\FLECHT~1\AppData\Local\Temp\tmp9mt0pqru



In [13]:
pulpo_worker.save_results(choices=choices, demand=demand, name='global_ammonia_showcase.xlsx')

In [14]:
pulpo_worker.summarize_results(choices=choices, demand=demand, zeroes=True)

The following demand / functional unit has been specified: 


,Demand
new market for ammonia | ammonia | GLO,1



These are the impacts contained in the objective:


,Key,Value
0,"('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2')",-2.969939



The following choices were made: 
biogas


...

biomethane


biomethane  \
                                                                                        Process   
Process 0  biogas upgrading to biomethane, chemical scrubbing w/ CCS | biomethane, 24 bar | GLO   

                                   
               Capacity     Value  
Process 0  1.000000e+10  0.628083

ammonia


ammonia  \
                                                                                                                        Process   
Process 0  ammonia production, steam methane reforming of biomethane, with CCS | ammonia production, biomethane, with CCS | GLO   

                                   
               Capacity     Value  
Process 0  1.000000e+10  1.000033

No additional constraints have been passed.
